# Mapping data
This is an alternative to using the google maps api, it requires no API key and no credit card.  The documentation for installing is here:  https://ipyleaflet.readthedocs.io/en/latest/installation.html




In [27]:
from ipyleaflet import Map, Marker, Icon, CircleMarker

center = (52.204793, 360.121558)

m = Map(center=center, zoom=15)

marker = Marker(location=center, draggable=False)
m.add_layer(marker);

m

Map(center=[52.204793, 360.121558], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

for some reason the default marker appears broken at the moment.  See below for a couple of work arounds.

In [44]:
import pandas as pd

stations = pd.read_csv("../Data/bikeshare_stations.csv")


In [45]:
locations = list(zip(stations.latitude, stations.longitude))
dc_center = (38.9072, -77.0369)
dcmap = Map(center=dc_center, zoom=12)

In [46]:
icon = Icon(icon_url="https://runestone.academy/runestone/static/marker.png")
for loc in locations:
    marker = CircleMarker(location=loc, radius=2)
    #marker= Marker(location=loc, draggable=False, icon=icon)
    dcmap.add_layer(marker)
    
dcmap

Map(center=[38.9072, -77.0369], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…